In [1]:
import os
import modules.util as util
import math
import numpy as np
import modules.datagenerator_test as dtgen
import modules.network_PoolerCLS as network
import modules.loss as loss
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow import keras

2023-03-09 08:00:03.000607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_testset')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [3]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'test': np.arange(math.floor(len(pairs)))}

# Generators
# test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=16)
test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=1)

In [4]:
print(f"Number of Test Pairs: {len(partition['test'])}")

Number of Test Pairs: 1320


In [5]:
siamese_obj = network.SiameseNetwork()
siamese_network = siamese_obj.make_siamese_net()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [6]:
siamese_network.compile(loss= loss.loss(1), optimizer=tf.keras.optimizers.Adam(learning_rate=0.5e-3), # 0.5e-6
                    metrics=["accuracy"])

In [7]:
# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Unrestored model, accuracy: {:5.2f}%".format(100 * acc))


1320/1320 - 71s - loss: 0.2572 - accuracy: 0.5174 - 71s/epoch - 54ms/step
Unrestored model, accuracy: 51.74%


In [8]:
checkpoint_path = os.path.join(home, 'checkpoints')

# Loads the weights
siamese_network.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

2023-03-09 08:01:34.395781: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/checkpoints: FAILED_PRECONDITION: /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/checkpoints; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


1320/1320 [==============================] - 67s 51ms/step - loss: 0.2289 - accuracy: 0.7038
Restored model, accuracy: 70.38%


In [9]:
predictions = siamese_network.predict(test_generator)

1320/1320 [==============================] - 75s 52ms/step


In [10]:
predictions.shape

(1320, 1)

In [11]:
test_generator.labels

[['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp',
  0,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-042/W-042-24.bmp',
  1,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  0,
  70,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1mar

In [12]:
labels = []
for i in test_generator.labels:
    # print(i[2])
    labels.append(i[2])

In [13]:
len(labels)

1320

In [14]:
predictions.shape

(1320, 1)

In [15]:
predictions[3][0]

0.13679118

In [16]:
pred_lbl=[]
for item in predictions:
    print(item[0])
    pred_lbl.append(1 if item[0]>=0.5 else 0)

0.99508876
0.7719584
0.025759095
0.13679118
0.35324275
0.7264784
0.65182894
0.016718699
0.82080114
0.07837224
0.4692439
0.99772066
0.017381698
0.97084177
0.4626242
0.9519128
0.11478162
0.06662821
0.35324275
0.7980327
0.19833454
0.11136004
0.90205944
0.99984777
0.70931673
0.9252606
0.098768644
0.0009086364
0.03669647
0.99291706
0.0184869
0.96569055
0.82716274
0.9960433
0.2506053
0.0729687
0.6495541
0.048428543
0.025829244
0.68360204
0.12912509
0.9871409
0.12912509
0.3783958
0.43763953
0.4413571
0.1491713
0.9068593
0.22727223
0.9972065
0.20541658
0.10363348
0.09440925
0.99983907
0.03791839
0.927773
0.067087494
0.04678939
0.20130545
0.22250564
0.6838048
0.97523093
0.116165474
0.8647758
0.21843642
0.6185025
0.73312825
0.62023675
0.20999941
0.8532933
0.18074857
0.99991703
0.63844925
0.9991979
0.30046272
0.75276464
0.33984095
0.6729366
0.29298863
0.9986719
0.032341797
0.7275909
0.1664308
0.9942194
0.66554314
0.91740024
0.9907494
0.999982
0.910231
0.5616769
0.14426625
0.97443944
0.8957457
0.9

In [17]:
pred_lbl

[1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,


In [18]:
# pred_lbl_flattened = [val for sublist in pred_lbl for val in sublist]

In [19]:
# pred_lbl_flattened

In [20]:
labels

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,


In [34]:
wrong_ls=[]
for i in range(len(pred_lbl)):
    # print(pred_lbl[i])
    # print(labels[i])
    if (pred_lbl[i] !=labels[i]):
        wrong_ls.append(i)

In [35]:
wrong_ls

[0,
 3,
 6,
 7,
 8,
 9,
 17,
 21,
 22,
 24,
 27,
 32,
 35,
 36,
 37,
 43,
 45,
 51,
 57,
 59,
 60,
 66,
 72,
 84,
 86,
 88,
 92,
 95,
 97,
 99,
 100,
 101,
 103,
 104,
 107,
 108,
 109,
 112,
 114,
 116,
 117,
 118,
 121,
 123,
 124,
 130,
 131,
 134,
 136,
 141,
 149,
 150,
 152,
 158,
 160,
 161,
 164,
 165,
 167,
 169,
 171,
 173,
 176,
 177,
 178,
 180,
 181,
 183,
 185,
 187,
 191,
 195,
 196,
 202,
 203,
 207,
 210,
 222,
 228,
 229,
 233,
 234,
 243,
 258,
 262,
 263,
 264,
 265,
 267,
 268,
 271,
 275,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 291,
 297,
 301,
 305,
 307,
 317,
 324,
 332,
 337,
 339,
 340,
 349,
 353,
 359,
 360,
 365,
 368,
 369,
 370,
 372,
 379,
 381,
 382,
 383,
 391,
 392,
 394,
 397,
 398,
 401,
 406,
 413,
 429,
 431,
 432,
 447,
 450,
 452,
 453,
 455,
 457,
 459,
 461,
 469,
 470,
 482,
 483,
 487,
 491,
 493,
 494,
 495,
 498,
 503,
 504,
 505,
 507,
 508,
 509,
 511,
 517,
 520,
 525,
 529,
 531,
 534,
 539,
 549,
 550,
 555,
 575,
 578,
 587,
 591

In [36]:
len(wrong_ls)

391

In [37]:
(1320-391)/1320

0.7037878787878787

In [38]:
len(test_generator.labels)

1320

In [40]:
photos = []
for item in wrong_ls:
    # print(item)
    photos.append(test_generator.labels[item])

In [42]:
len(photos)

391

In [41]:
photos

[['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp',
  0,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-044/W-044-13.bmp',
  1,
  224,
  70],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-20.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-09.bmp',
  0,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1mar

In [44]:
photos[0][0]

'/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp'

In [45]:
photos[0][1]

'/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_1margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp'

In [55]:
scarf = 0
sunglasses = 0
normal = 0
yawn = 0
for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        scarf+=1
    if root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        scarf+=1
    if root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        scarf+=1
    if root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        scarf+=1

    # SUNGLASSES
    if root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        sunglasses+=1
    if root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        sunglasses+=1
    if root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        sunglasses+=1
    if root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        sunglasses+=1
    
    # YAWN
    if root1.endswith("04") or root1.endswith("17"):
        yawn+=1
    if root2.endswith("04") or root2.endswith("17"):
        yawn+=1


In [60]:
print(f"Scarf Total: {scarf} Percentage: {(scarf/(391*2))*100}")
print(f"Sunglasses Total: {sunglasses} Percentage: {(sunglasses/(391*2))*100}")
print(f"Yawn Total: {yawn} Percentage: {(yawn/(391*2))*100}")
print(f"Others: {(391*2)-scarf-sunglasses-yawn} Percentage: {(((391*2)-scarf-sunglasses-yawn)/(391*2))*100}")

Scarf Total: 198 Percentage: 25.31969309462916
Sunglasses Total: 178 Percentage: 22.762148337595907
Yawn Total: 53 Percentage: 6.7774936061381075
Others: 353 Percentage: 45.14066496163683


In [66]:
scarf = 0
sunglasses = 0
normal = 0
yawn = 0
for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        scarf+=1
    elif root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        scarf+=1
    elif root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        scarf+=1
    elif root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        scarf+=1

    # SUNGLASSES
    if root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        sunglasses+=1
    elif root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        sunglasses+=1
    elif root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        sunglasses+=1
    elif root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        sunglasses+=1
    
    # YAWN
    if root1.endswith("04") or root1.endswith("17"):
        yawn+=1
    elif root2.endswith("04") or root2.endswith("17"):
        yawn+=1

In [68]:
print(f"Scarf Total: {scarf} Percentage: {(scarf/(391))*100}")
print(f"Sunglasses Total: {sunglasses} Percentage: {(sunglasses/(391))*100}")
print(f"Yawn Total: {yawn} Percentage: {(yawn/(391))*100}")

Scarf Total: 177 Percentage: 45.26854219948849
Sunglasses Total: 154 Percentage: 39.38618925831202
Yawn Total: 51 Percentage: 13.043478260869565


In [70]:
obstruction =0
normal = 0

for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        obstruction+=1
    elif root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        obstruction+=1
    elif root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        obstruction+=1
    elif root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        obstruction+=1
    # SUNGLASSES
    elif root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        obstruction+=1
    elif root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        obstruction+=1
    elif root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        obstruction+=1
    elif root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        obstruction+=1
    # YAWN
    elif root1.endswith("04") or root1.endswith("17"):
        obstruction+=1
    elif root2.endswith("04") or root2.endswith("17"):
        obstruction+=1
    else:
        normal+=1

In [71]:
print(f"Obstruction Total: {obstruction} Percentage: {(obstruction/(391))*100}")
print(f"Normal Total: {normal} Percentage: {(normal/(391))*100}")

Obstruction Total: 312 Percentage: 79.79539641943734
Normal Total: 79 Percentage: 20.20460358056266


In [ ]:
scarf_vs_scarf = 0
scarf_vs_sunglasses = 0
scarf_vs_normal = 0
sunglasses_vs_normal = 0
yawn_vs_yawn = 0